# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [56]:
# import libraries
import re
import pandas as pd
import numpy as np
import pickle

# nltk libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger','stopwords'])
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

# machine learning libraries
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import classification_report

# sql libraries
from sqlalchemy import create_engine

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql ('SELECT * FROM Messages', engine)
X = df['message']
y = df.iloc[:,4:]

In [3]:
df.head(3)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [11]:
def tokenize(text):
    # convert text to lowercase and remove punctuation characters
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # store the stop words in cache so function runs faster
    # create set of stop words because a set is faster to iterate over than a list
    cachedStopWords = set(stopwords.words('english'))

    
    # remove stop words
    tokens = [w for w in tokens if w not in cachedStopWords]
    
    # instantiate lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # lemmatize tokens
    clean_tokens = [lemmatizer.lemmatize(w) for w in tokens]

    return clean_tokens

In [12]:
# test tokenize function on first message
tokenize(X.loc[0])

['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']

In [13]:
# test tokenize function on second message
tokenize(X.loc[1])

['hurricane']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [14]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

# train classifier
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [19]:
sample_msg = ['There was an explosion and there is a large fire. People are severly injured.']

y_pred_sample = pipeline.predict(sample_msg)

print(y_train.columns.values[(y_pred_sample.flatten()==1)])

['related' 'aid_related' 'medical_help' 'weather_related']


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [22]:
y_pred_test = pipeline.predict(X_test)

# Print classification report on test data
print(classification_report(y_test.values, y_pred_test, target_names=y.columns.values))

model_accuracy = (y_pred_test == y_test.values).mean()
print('The model accuracy is {:.3f}'.format(model_accuracy))

                        precision    recall  f1-score   support

               related       0.86      0.92      0.89      5060
               request       0.80      0.43      0.56      1133
                 offer       0.00      0.00      0.00        28
           aid_related       0.76      0.60      0.67      2796
          medical_help       0.51      0.08      0.13       537
      medical_products       0.66      0.08      0.15       321
     search_and_rescue       0.71      0.03      0.06       168
              security       0.00      0.00      0.00       124
              military       0.45      0.06      0.11       234
                 water       0.88      0.25      0.39       431
                  food       0.85      0.40      0.54       754
               shelter       0.77      0.30      0.43       586
              clothing       0.67      0.10      0.18       115
                 money       0.50      0.02      0.05       165
        missing_people       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [50]:
# get parameters of transformers in our pipeline
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fe79d8061e0>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [29]:
grid_values = {'tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [10, 20]}

# set cv=3 to perform 3-fold cross validation which allows gridsearch to run faster
# higher fold cross validation causes gridsearch to run slower
cv = GridSearchCV (pipeline, param_grid=grid_values, scoring='f1_micro', cv=3, n_jobs=8)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [30]:
cv.fit(X_train, y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=8,
       param_grid={'tfidf__use_idf': (True, False), 'clf__estimator__n_estimators': [10, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_micro', verbose=0)

In [32]:
y_pred_test = cv.predict(X_test)

# Print classification report on test data
print(classification_report(y_test.values, y_pred_test, target_names=y.columns.values))

model_accuracy = (y_pred_test == y_test.values).mean()
print('The model accuracy is {:.3f}'.format(model_accuracy))

                        precision    recall  f1-score   support

               related       0.85      0.95      0.90      5060
               request       0.82      0.49      0.61      1133
                 offer       0.00      0.00      0.00        28
           aid_related       0.77      0.65      0.70      2796
          medical_help       0.59      0.08      0.14       537
      medical_products       0.69      0.07      0.13       321
     search_and_rescue       0.33      0.03      0.05       168
              security       0.50      0.01      0.02       124
              military       0.68      0.06      0.12       234
                 water       0.86      0.32      0.47       431
                  food       0.83      0.60      0.70       754
               shelter       0.82      0.33      0.47       586
              clothing       0.62      0.13      0.22       115
                 money       1.00      0.02      0.04       165
        missing_people       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [33]:
cv.best_params_

{'clf__estimator__n_estimators': 20, 'tfidf__use_idf': False}

GridSearch took a over 20 minutes to run and it would be worthwhile to look into improving the speed of GridSearch. When calling `best_params_` we can see that GridSearch found that the model performed best when n_estimators was set to 20 and when tfidf was not used.

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

Let's start by redefining our tokenize function and we will then build a custom transformer called `StartingVerbExtractor` which extracts the starting verb of a sentence.

In [46]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [47]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

Let's now create a new pipeline which utilizes `FeatureUnion` which allows us to run multiple steps in parallel and then combine their results before passing them into the next step. The new pipeline will run the `text_pipeline` and the `starting_verb` transformer in parallel, combine their results, and then pass them into the classifier.

Instead of using a **RandomForest** classifier as we did earlier, we are going to try the **AdaBoost** classifier and for the **TfidfTransformer** we will set `use_idf` to `False` since that was the optimal parameter found by GridSearch.

In [52]:
def model_pipeline():
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)), # using updated tokenize function
                ('tfidf', TfidfTransformer(use_idf=False)) # false setting from GridSearch
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])

    return pipeline

In [53]:
model = model_pipeline()

model.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

In [54]:
y_pred_test = model.predict(X_test)

# Print classification report on test data
print(classification_report(y_test.values, y_pred_test, target_names=y.columns.values))

model_accuracy = (y_pred_test == y_test.values).mean()
print('The model accuracy is {:.3f}'.format(model_accuracy))

                        precision    recall  f1-score   support

               related       0.84      0.95      0.89      5060
               request       0.79      0.55      0.65      1133
                 offer       0.33      0.04      0.06        28
           aid_related       0.77      0.60      0.67      2796
          medical_help       0.57      0.23      0.32       537
      medical_products       0.60      0.31      0.41       321
     search_and_rescue       0.53      0.17      0.26       168
              security       0.21      0.03      0.06       124
              military       0.57      0.28      0.38       234
                 water       0.77      0.61      0.68       431
                  food       0.82      0.67      0.74       754
               shelter       0.76      0.55      0.64       586
              clothing       0.65      0.38      0.48       115
                 money       0.62      0.24      0.35       165
        missing_people       0.56      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


This final model produced a higher overall average **F1 score** compared to the previous two models.

### 9. Export your model as a pickle file

In [58]:
file_name = 'classifier.pkl'
with open (file_name, 'wb') as f:
    pickle.dump(model, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.